<a href="https://colab.research.google.com/github/Maitreyi1506/Protein-SubChloroplast-Localisation/blob/main/Novel%20Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

# Importing dataset

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Novel Dataset:

In [ ]:
novel_df = pd.read_csv("/content/gdrive/MyDrive/Dataset/Novel_BinaryML.csv")
novel_df.head()

,PDBid,Sequence,envelope,lumen,plastoglobule,stroma,thylakoid_membrane
0,A1JHN0,MDLCSSTGRGACLSPASTSRPCPAPVHLRGRRLAFSPAQPAGRRHL...,1,0,0,0,0
1,A1L4X0,MDTVLMATTPPIRCLHASIPTVFRSPAIYQVSCRSSQLFSYRSTTM...,1,0,0,0,0
2,A2XLM6,MSSSSLVTSLLFSSSSSSNTATSTSSRRSFSLFSKNQYCKPSPLRR...,1,0,0,0,0
3,A8MPR5,MACRFPLHSSSPSQFLSPENRQRLPRNYPSISCQNNSATNVVHEDG...,1,0,0,0,0
4,A8MS68,MQSAMALSFSQTSFTRPNHVLGSSGSVFSTPRSLRFCGLRREAFGF...,0,0,0,1,0


In [ ]:
def parse_hmm(fname):
    f = open(fname)
    line=f.readline()
    while line[0]!='#':
        line=f.readline()
    f.readline()
    f.readline()
    f.readline()
    f.readline()
    seq = []
    extras = np.zeros([0,10])
    prob = np.zeros([0,20])
    line = f.readline()
    while line[0:2]!='//':
        lineinfo = line.split()
        seq.append(lineinfo[0])
        probs_ = [2**(-float(lineinfo[i])/1000) if lineinfo[i]!='*' else 0. for i in range(2,22)]
        prob = np.concatenate((prob,np.matrix(probs_)),axis=0)

        line = f.readline()
        lineinfo = line.split()
        extras_ = [2**(-float(lineinfo[i])/1000) if lineinfo[i]!='*' else 0. for i in range(0,10)]
        extras = np.concatenate((extras,np.matrix(extras_)),axis=0)

        line = f.readline()
        assert len(line.strip())==0

        line = f.readline()
    return (''.join(seq),prob,extras)

In [ ]:
import os
os.chdir('/content/gdrive/MyDrive/Dataset/Novel_HMM')
files = os.listdir()

In [ ]:
df_parsed = pd.DataFrame(columns=['Sequence', 'prob_hmm','extras_hmm'])
for file in files:
  row = parse_hmm(file)
  row_dict = dict(zip(df_parsed.columns, row))
  df_parsed = df_parsed.append(row_dict, ignore_index = True)

df_parsed.head()

In [ ]:
df_parsed.head()

,Sequence,prob_hmm,extras_hmm
0,MMARFVSVSSCQFHFGFREVSPPSVTSYPRRFEVSDRRFPAIPIKC...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...
1,MAPVPVSVSATLAPPPAAPPKTTSRSWERRAPADAAFAAASSVAGS...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...
2,MSSMNFNPFQNWFEKPPNPVPSINFVSLADSFFPKSQSPNFASIGL...,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...
3,MATVTILSPKSIPKVTDSKFGARVSDQIVNVVKCGKSGRRLKLAKL...,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. 0...
4,MACRFPLHSSSPSQFLSPENRQRLPRNYPSISCQNNSATNVVHEDG...,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...


In [ ]:
merged_df = novel_df.merge(df_parsed, on='Sequence', how='inner')

In [ ]:
merged_df.head()

,PDBid,Sequence,envelope,lumen,plastoglobule,stroma,thylakoid_membrane,prob_hmm,extras_hmm
0,A1JHN0,MDLCSSTGRGACLSPASTSRPCPAPVHLRGRRLAFSPAQPAGRRHL...,1,0,0,0,0,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...
1,A1L4X0,MDTVLMATTPPIRCLHASIPTVFRSPAIYQVSCRSSQLFSYRSTTM...,1,0,0,0,0,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...
2,A2XLM6,MSSSSLVTSLLFSSSSSSNTATSTSSRRSFSLFSKNQYCKPSPLRR...,1,0,0,0,0,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...
3,A8MPR5,MACRFPLHSSSPSQFLSPENRQRLPRNYPSISCQNNSATNVVHEDG...,1,0,0,0,0,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...
4,A8MS68,MQSAMALSFSQTSFTRPNHVLGSSGSVFSTPRSLRFCGLRREAFGF...,0,0,0,1,0,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...


In [ ]:
def parse_pssm(fname):
    f = open(fname)
    # the 4th line should be the start of the PSSM data
    f.readline()
    f.readline()
    f.readline()
    seq = []
    lprob = np.zeros([0,20])
    prob = np.zeros([0,20])
    extra = np.zeros([0,2])
    line = f.readline()
    while len(line.strip())>0:
        lineinfo = line.split()
        seq.append(lineinfo[1])
        lprobs_ = [float(lineinfo[i]) for i in range(2,22)]
        lprob = np.concatenate((lprob,np.matrix(lprobs_)),axis=0)
        probs_ = [float(lineinfo[i])/100 for i in range(22,42)]
        prob = np.concatenate((prob,np.matrix(probs_)),axis=0)
        extras_ = [float(lineinfo[i]) for i in range(42,44)]
        extra = np.concatenate((extra,np.matrix(extras_)),axis=0)
        line = f.readline()

    return (''.join(seq),prob,lprob,extra)

In [ ]:
os.chdir('/content/gdrive/MyDrive/Dataset/Novel_PSSM')
files = os.listdir()

In [ ]:
df_parsed = pd.DataFrame(columns=['Sequence', 'prob_pssm','lprob_pssm','extras_pssm'])
for file in files:
  row = parse_pssm(file)
  row_dict = dict(zip(df_parsed.columns, row))
  df_parsed = df_parsed.append(row_dict, ignore_index = True)

df_parsed.head()

In [ ]:
df_parsed.head()

,Sequence,prob_pssm,lprob_pssm,extras_pssm
0,MMARFVSVSSCQFHFGFREVSPPSVTSYPRRFEVSDRRFPAIPIKC...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -3. -3. -4. -2. -2. -3. -4. -3. 1. ...,"[[[[[0.99 0.21]]]], [[[[2.01 0.59]]]], [[[[0.9..."
1,MAPVPVSVSATLAPPPAAPPKTTSRSWERRAPADAAFAAASSVAGS...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -3. -4. -5. -3. -2. -4. -4. -3. 1. ...,"[[[[[1.35 0.35]]]], [[[[0.63 0.27]]]], [[[[0.5..."
2,MSSMNFNPFQNWFEKPPNPVPSINFVSLADSFFPKSQSPNFASIGL...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-3. -4. -5. -6. -4. -3. -4. -5. -4. -1. ...,"[[[[[2.44 0.56]]]], [[[[0.95 0.31]]]], [[[[1.4..."
3,MATVTILSPKSIPKVTDSKFGARVSDQIVNVVKCGKSGRRLKLAKL...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -3. -4. -4. -2. -2. -3. -4. -3. 1. ...,"[[[[[0.76 0.19]]]], [[[[0.79 0.28]]]], [[[[0.4..."
4,MACRFPLHSSSPSQFLSPENRQRLPRNYPSISCQNNSATNVVHEDG...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-3. -3. -4. -5. -3. -2. -4. -5. -4. -1. ...,"[[[[[2.05 0.4 ]]]], [[[[0.73 0.21]]]], [[[[1. ..."


In [ ]:
novel_df_final = merged_df.merge(df_parsed, on='Sequence', how='inner')
novel_df_final.head()

,PDBid,Sequence,envelope,lumen,plastoglobule,stroma,thylakoid_membrane,prob_hmm,extras_hmm,prob_pssm,lprob_pssm,extras_pssm
0,A1JHN0,MDLCSSTGRGACLSPASTSRPCPAPVHLRGRRLAFSPAQPAGRRHL...,1,0,0,0,0,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-1. -2. -3. -4. -2. -2. -3. -4. -3. 2. ...,"[[[[[0.4 0.04]]]], [[[[0.48 0.08]]]], [[[[0.3..."
1,A1L4X0,MDTVLMATTPPIRCLHASIPTVFRSPAIYQVSCRSSQLFSYRSTTM...,1,0,0,0,0,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-1. -2. -3. -4. -2. -1. -2. -3. -2. 1. ...,"[[[[[0.57 0.06]]]], [[[[0.92 0.06]]]], [[[[0.4..."
2,A2XLM6,MSSSSLVTSLLFSSSSSSNTATSTSSRRSFSLFSKNQYCKPSPLRR...,1,0,0,0,0,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -3. -4. -5. -3. -2. -4. -4. -3. 0. ...,"[[[[[1.56 0.3 ]]]], [[[[0.51 0.14]]]], [[[[0.4..."
3,A8MPR5,MACRFPLHSSSPSQFLSPENRQRLPRNYPSISCQNNSATNVVHEDG...,1,0,0,0,0,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-3. -3. -4. -5. -3. -2. -4. -5. -4. -1. ...,"[[[[[2.05 0.4 ]]]], [[[[0.73 0.21]]]], [[[[1. ..."
4,A8MS68,MQSAMALSFSQTSFTRPNHVLGSSGSVFSTPRSLRFCGLRREAFGF...,0,0,0,1,0,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -3. -3. -4. -2. -2. -3. -4. -3. 0. ...,"[[[[[1. 0.17]]]], [[[[0.55 0.11]]]], [[[[0.4..."


In [ ]:
novel_df_final['to_predict'] = novel_df_final[['envelope', 'lumen', 'plastoglobule', 'stroma', 'thylakoid_membrane']].apply(list, axis=1)

In [ ]:
novel_df_final.head()

,PDBid,Sequence,envelope,lumen,plastoglobule,stroma,thylakoid_membrane,prob_hmm,extras_hmm,prob_pssm,lprob_pssm,extras_pssm,to_predict
0,A1JHN0,MDLCSSTGRGACLSPASTSRPCPAPVHLRGRRLAFSPAQPAGRRHL...,1,0,0,0,0,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-1. -2. -3. -4. -2. -2. -3. -4. -3. 2. ...,"[[[[[0.4 0.04]]]], [[[[0.48 0.08]]]], [[[[0.3...","[1, 0, 0, 0, 0]"
1,A1L4X0,MDTVLMATTPPIRCLHASIPTVFRSPAIYQVSCRSSQLFSYRSTTM...,1,0,0,0,0,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-1. -2. -3. -4. -2. -1. -2. -3. -2. 1. ...,"[[[[[0.57 0.06]]]], [[[[0.92 0.06]]]], [[[[0.4...","[1, 0, 0, 0, 0]"
2,A2XLM6,MSSSSLVTSLLFSSSSSSNTATSTSSRRSFSLFSKNQYCKPSPLRR...,1,0,0,0,0,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -3. -4. -5. -3. -2. -4. -4. -3. 0. ...,"[[[[[1.56 0.3 ]]]], [[[[0.51 0.14]]]], [[[[0.4...","[1, 0, 0, 0, 0]"
3,A8MPR5,MACRFPLHSSSPSQFLSPENRQRLPRNYPSISCQNNSATNVVHEDG...,1,0,0,0,0,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-3. -3. -4. -5. -3. -2. -4. -5. -4. -1. ...,"[[[[[2.05 0.4 ]]]], [[[[0.73 0.21]]]], [[[[1. ...","[1, 0, 0, 0, 0]"
4,A8MS68,MQSAMALSFSQTSFTRPNHVLGSSGSVFSTPRSLRFCGLRREAFGF...,0,0,0,1,0,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -3. -3. -4. -2. -2. -3. -4. -3. 0. ...,"[[[[[1. 0.17]]]], [[[[0.55 0.11]]]], [[[[0.4...","[0, 0, 0, 1, 0]"


In [ ]:
novel_df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121 entries, 0 to 120
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   PDBid               121 non-null    object
 1   Sequence            121 non-null    object
 2   envelope            121 non-null    int64 
 3   lumen               121 non-null    int64 
 4   plastoglobule       121 non-null    int64 
 5   stroma              121 non-null    int64 
 6   thylakoid_membrane  121 non-null    int64 
 7   prob_hmm            121 non-null    object
 8   extras_hmm          121 non-null    object
 9   prob_pssm           121 non-null    object
 10  lprob_pssm          121 non-null    object
 11  extras_pssm         121 non-null    object
 12  to_predict          121 non-null    object
dtypes: int64(5), object(8)
memory usage: 13.2+ KB


# Data Preprocessing and NLP

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
novel_df_final['tokens'] = novel_df_final['Sequence'].apply(list)

In [ ]:
def get_elmo_embeddings(df, column_name):
  elmo = hub.load("https://tfhub.dev/google/elmo/3")
  # elmo_embeddings = elmo(df[column_name], signature="default", as_dict=True)["elmo"]
  elmo_embeddings = []
  for token in df[column_name]:
    elmo_embedding = elmo.signatures["default"](tf.constant(token))["elmo"]
    elmo_embeddings.append(elmo_embedding)
  df["ELMo"] = elmo_embeddings
  return df

In [ ]:
novel_df_final = get_elmo_embeddings(novel_df_final, 'tokens')
novel_df_final.head()

,PDBid,Sequence,envelope,lumen,plastoglobule,stroma,thylakoid_membrane,prob_hmm,extras_hmm,prob_pssm,lprob_pssm,extras_pssm,to_predict,tokens,ELMo
0,A1JHN0,MDLCSSTGRGACLSPASTSRPCPAPVHLRGRRLAFSPAQPAGRRHL...,1,0,0,0,0,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-1. -2. -3. -4. -2. -2. -3. -4. -3. 2. ...,"[[[[[0.4 0.04]]]], [[[[0.48 0.08]]]], [[[[0.3...","[1, 0, 0, 0, 0]","[M, D, L, C, S, S, T, G, R, G, A, C, L, S, P, ...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
1,A1L4X0,MDTVLMATTPPIRCLHASIPTVFRSPAIYQVSCRSSQLFSYRSTTM...,1,0,0,0,0,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-1. -2. -3. -4. -2. -1. -2. -3. -2. 1. ...,"[[[[[0.57 0.06]]]], [[[[0.92 0.06]]]], [[[[0.4...","[1, 0, 0, 0, 0]","[M, D, T, V, L, M, A, T, T, P, P, I, R, C, L, ...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
2,A2XLM6,MSSSSLVTSLLFSSSSSSNTATSTSSRRSFSLFSKNQYCKPSPLRR...,1,0,0,0,0,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. 0. 0. 0. 0.5 1. 1. ]]...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -3. -4. -5. -3. -2. -4. -4. -3. 0. ...,"[[[[[1.56 0.3 ]]]], [[[[0.51 0.14]]]], [[[[0.4...","[1, 0, 0, 0, 0]","[M, S, S, S, S, L, V, T, S, L, L, F, S, S, S, ...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
3,A8MPR5,MACRFPLHSSSPSQFLSPENRQRLPRNYPSISCQNNSATNVVHEDG...,1,0,0,0,0,[[[[[0. 0. 0. 0. ...,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0....,[[[[[-3. -3. -4. -5. -3. -2. -4. -5. -4. -1. ...,"[[[[[2.05 0.4 ]]]], [[[[0.73 0.21]]]], [[[[1. ...","[1, 0, 0, 0, 0]","[M, A, C, R, F, P, L, H, S, S, S, P, S, Q, F, ...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
4,A8MS68,MQSAMALSFSQTSFTRPNHVLGSSGSVFSTPRSLRFCGLRREAFGF...,0,0,0,1,0,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0....,[[[[[1. 0. 0. 0. ...,[[[[[0. 0. 0. 0. 0. 0. 0. 0. 0...,[[[[[-2. -3. -3. -4. -2. -2. -3. -4. -3. 0. ...,"[[[[[1. 0.17]]]], [[[[0.55 0.11]]]], [[[[0.4...","[0, 0, 0, 1, 0]","[M, Q, S, A, M, A, L, S, F, S, Q, T, S, F, T, ...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."


# Converting to tensor

In [ ]:
novel_df_final['ELMo_tensor'] = np.zeros(len(novel_df_final['ELMo']))

In [ ]:
ELMo_tensor_list = []

for i in range(len(novel_df_final['ELMo'])):
  array = np.array(novel_df_final['ELMo'][i], dtype=float)
  ELMo_tensor = tf.convert_to_tensor(array, dtype=tf.float32)
  ELMo_tensor_list.append(ELMo_tensor)

novel_df_final['ELMo_tensor'] = ELMo_tensor_list
# P = tf.stack(prob_hmm_tensor_list)

In [ ]:
novel_df_final['ELMo_tensor']

0      (((tf.Tensor(-0.4847446, shape=(), dtype=float...
1      (((tf.Tensor(-0.4847446, shape=(), dtype=float...
2      (((tf.Tensor(-0.4847446, shape=(), dtype=float...
3      (((tf.Tensor(-0.4847446, shape=(), dtype=float...
4      (((tf.Tensor(-0.4847446, shape=(), dtype=float...
                             ...                        
116    (((tf.Tensor(-0.4847446, shape=(), dtype=float...
117    (((tf.Tensor(-0.4847446, shape=(), dtype=float...
118    (((tf.Tensor(-0.4847446, shape=(), dtype=float...
119    (((tf.Tensor(-0.4847446, shape=(), dtype=float...
120    (((tf.Tensor(-0.4847446, shape=(), dtype=float...
Name: ELMo_tensor, Length: 121, dtype: object

# Training RNN model


In [ ]:
novel_df_final['to_predict'] = novel_df_final['to_predict'].apply(np.array)
novel_df_final['prob_hmm'] = novel_df_final['prob_hmm'].apply(np.array)
novel_df_final['extras_hmm'] = novel_df_final['extras_hmm'].apply(np.array)
novel_df_final['prob_pssm'] = novel_df_final['prob_pssm'].apply(np.array)
novel_df_final['lprob_pssm'] = novel_df_final['lprob_pssm'].apply(np.array)
novel_df_final['extras_pssm'] = novel_df_final['extras_pssm'].apply(np.array)
novel_df_final['ELMo'] = novel_df_final['ELMo'].apply(np.array)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = novel_df_final.loc[:, ['prob_hmm', 'extras_hmm','prob_pssm','lprob_pssm','extras_pssm','ELMo_tensor']]
y = novel_df_final['to_predict']
# y = novel_df_final['envelope']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [ ]:
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers import  Dense, Embedding, LSTM, Bidirectional

In [ ]:
model = Sequential()
model.add(Embedding(10000, 128, input_length=10000))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
X_train

,prob_hmm,extras_hmm,prob_pssm,lprob_pssm,extras_pssm,ELMo_tensor
102,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.09532510332851775...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0714956...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-2.0, -2.0, -3.0, -4.0, -2.0, -1.0, -3.0, -4...","[[0.99, 0.13], [0.6, 0.09], [1.86, 0.11], [0.3...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
40,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3328629...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-3.0, -3.0, -4.0, -5.0, -3.0, -2.0, -4.0, -5...","[[1.95, 0.43], [0.96, 0.27], [0.44, 0.17], [0....","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
35,"[[0.0, 0.0, 0.0, 0.024740183308477246, 0.0, 0....","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0543717...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-4.0, -5.0, -6.0, -7.0, -5.0, -4.0, -6.0, -6...","[[3.15, 0.82], [1.03, 0.42], [1.62, 0.49], [0....","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
116,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1094239...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-4.0, -4.0, -5.0, -6.0, -4.0, -3.0, -5.0, -5...","[[2.64, 0.71], [0.32, 0.2], [1.48, 0.43], [0.8...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
9,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-3.0, -4.0, -4.0, -5.0, -4.0, -3.0, -4.0, -5...","[[2.15, 0.43], [0.64, 0.18], [0.6, 0.17], [1.0...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
...,...,...,...,...,...,...
73,"[[0.0, 0.0, 0.0, 0.395294558301972, 0.0, 0.0, ...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3384465...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-3.0, -3.0, -4.0, -5.0, -3.0, -2.0, -4.0, -5...","[[1.73, 0.35], [0.54, 0.17], [0.52, 0.18], [0....","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
31,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0358222...","[[0.9125654885434519, 0.0, 0.08741350517115384...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-2.0, -3.0, -3.0, -4.0, -3.0, -2.0, -3.0, -4...","[[1.31, 0.2], [0.5, 0.09], [0.45, 0.09], [0.86...","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
89,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 1.0,...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-3.0, -4.0, -5.0, -6.0, -4.0, -3.0, -5.0, -5...","[[2.31, 0.51], [1.22, 0.39], [0.71, 0.28], [0....","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."
30,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.3624867...","[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...","[[-2.0, -3.0, -4.0, -5.0, -3.0, -2.0, -4.0, -4...","[[1.61, 0.25], [0.95, 0.16], [0.51, 0.12], [0....","(((tf.Tensor(-0.4847446, shape=(), dtype=float..."


In [ ]:
y_train

102    [0, 0, 0, 1, 0]
40     [0, 0, 0, 1, 0]
35     [1, 0, 0, 0, 0]
116    [0, 0, 0, 0, 1]
9      [1, 0, 0, 0, 0]
            ...       
73     [0, 0, 0, 0, 1]
31     [1, 0, 0, 0, 0]
89     [0, 0, 0, 0, 1]
30     [0, 1, 0, 0, 0]
43     [1, 0, 0, 0, 0]
Name: to_predict, Length: 90, dtype: object

In [ ]:
history=model.fit(X_train, y_train,
           batch_size=16,
           epochs=12,
           validation_data=[X_test, y_test])
print(history.history['loss'])
print(history.history['accuracy'])

Epoch 1/12
6/6 [==============================] - 5s 199ms/step - loss: 0.6939 - accuracy: 0.4444 - val_loss: 0.6933 - val_accuracy: 0.4839
Epoch 2/12
6/6 [==============================] - 0s 24ms/step - loss: 0.6924 - accuracy: 0.5333 - val_loss: 0.6936 - val_accuracy: 0.4839
Epoch 3/12
6/6 [==============================] - 0s 28ms/step - loss: 0.6910 - accuracy: 0.5778 - val_loss: 0.6940 - val_accuracy: 0.4839
Epoch 4/12
6/6 [==============================] - 0s 34ms/step - loss: 0.6904 - accuracy: 0.5667 - val_loss: 0.6941 - val_accuracy: 0.4839
Epoch 5/12
6/6 [==============================] - 0s 37ms/step - loss: 0.6894 - accuracy: 0.5667 - val_loss: 0.6944 - val_accuracy: 0.4839
Epoch 6/12
6/6 [==============================] - 0s 32ms/step - loss: 0.6879 - accuracy: 0.5667 - val_loss: 0.6950 - val_accuracy: 0.4839
Epoch 7/12
6/6 [==============================] - 0s 38ms/step - loss: 0.6864 - accuracy: 0.5667 - val_loss: 0.6953 - val_accuracy: 0.4839
Epoch 8/12
6/6 [==========

# Calculating accuracies

In [ ]:
y_test

29     [1, 0, 0, 0, 0]
3      [1, 0, 0, 0, 0]
17     [1, 0, 0, 0, 0]
22     [0, 0, 0, 0, 1]
41     [1, 0, 0, 0, 0]
51     [0, 0, 0, 0, 1]
65     [0, 0, 0, 0, 1]
26     [1, 0, 0, 0, 0]
80     [1, 0, 0, 0, 0]
76     [0, 0, 0, 0, 1]
99     [1, 0, 0, 0, 0]
15     [0, 0, 0, 1, 0]
71     [0, 0, 0, 0, 1]
81     [1, 0, 0, 0, 0]
117    [1, 0, 0, 0, 0]
32     [1, 0, 0, 0, 0]
23     [0, 0, 0, 0, 1]
107    [0, 0, 0, 0, 1]
105    [0, 0, 0, 0, 1]
46     [0, 0, 0, 1, 0]
2      [1, 0, 0, 0, 0]
55     [0, 0, 0, 1, 1]
28     [1, 0, 0, 0, 0]
104    [0, 0, 0, 1, 0]
88     [1, 0, 0, 0, 0]
90     [0, 0, 0, 0, 1]
108    [0, 0, 0, 1, 1]
77     [1, 0, 0, 0, 0]
1      [1, 0, 0, 0, 0]
0      [1, 0, 0, 0, 0]
97     [1, 0, 0, 0, 0]
Name: to_predict, dtype: object

In [ ]:
y_pred = model.predict(X_test)
# y_pred = []

In [ ]:
y_pred

0     [1, 0, 0, 0, 0]
1     [0, 0, 0, 0, 1]
2     [0, 0, 0, 1, 1]
3     [0, 0, 0, 1, 1]
4     [1, 0, 0, 0, 0]
5     [0, 0, 0, 1, 0]
6     [0, 0, 0, 1, 1]
7     [1, 0, 0, 0, 0]
8     [1, 0, 0, 0, 0]
9     [0, 0, 0, 0, 1]
10    [0, 0, 0, 1, 1]
11    [0, 0, 0, 0, 1]
12    [1, 0, 0, 0, 0]
13    [0, 0, 0, 1, 0]
14    [0, 0, 0, 1, 0]
15    [0, 0, 0, 1, 0]
16    [0, 0, 0, 0, 1]
17    [0, 0, 0, 0, 1]
18    [0, 0, 0, 1, 1]
19    [1, 0, 0, 0, 0]
20    [0, 0, 0, 0, 1]
21    [1, 0, 0, 0, 0]
22    [0, 0, 0, 1, 1]
23    [0, 0, 0, 1, 0]
24    [0, 0, 0, 0, 1]
25    [1, 0, 0, 0, 0]
26    [1, 0, 0, 0, 0]
27    [0, 0, 0, 1, 0]
28    [1, 0, 0, 0, 0]
29    [1, 0, 0, 0, 0]
30    [1, 0, 0, 0, 0]
dtype: object

In [ ]:
#Normal Accuracy

import statistics

accuracy = []
for i,j in zip(y_test.index,y_pred.index):
  sum = 0
  lisA = y_test[i]
  lisB = y_pred[j]
  for k in range(5):
    if lisA[k]==lisB[k]:
      sum+=0.2
  accuracy.append(sum)

print("The normal accuracy is: ", statistics.mean(accuracy))

The normal accuracy is:  0.7290322580645162


In [ ]:
#Strict Accuracy

import statistics

accuracy = []
for i,j in zip(y_test.index,y_pred.index):
  if all(element in y_test[i] for element in y_pred[j]):
    accuracy.append(1)
  else:
    accuracy.append(0)

print("The strict accuracy is: ", statistics.mean(accuracy))

The strict accuracy is:  0.5384615384615384
